<a href="https://colab.research.google.com/github/Umapak/hendaiako_baserriak/blob/gh_viewers/hendaiako_baserriak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baserri_zerrendakeko datuak eta Python+Foliumekin egiten da mapa hemen 

In [1]:
%%capture
!pip install --upgrade gspread
!apt install libspatialindex-dev
!pip install rtree
!pip install geopandas

In [2]:
#@title Google drive-n dagoen 'baserri_zerrendak'-etik datuak zuzenean hartu
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
# hemen zehaztu behar da zein den google drive-n eman zaion izena
worksheet = gc.open('baserri_zerrendak').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

# datu sorta guztia taula erabilgarri formatura pasako du eta campo bakoitzaren izena jarriko du.
import pandas as pd
import geopandas as gpd
baserriak = pd.DataFrame.from_records(rows)
baserriak.columns = baserriak.iloc[0]
baserriak = baserriak.iloc[1: , :]

In [3]:
#@title Datuok aztertu eta txukundu
baserriak[['google_maps','izena']] = baserriak[['google_maps', 'izena']].astype('string')
baserriak['zenbakia'] = baserriak['zenbakia'].astype(str).astype(int)

In [4]:
baserriak.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 1 to 82
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   zenbakia         82 non-null     int64 
 1   kontrola         82 non-null     object
 2   izena            82 non-null     string
 3   google_maps      82 non-null     string
 4   auzoa            82 non-null     object
 5   zonaldea         82 non-null     object
 6   en_mapa          82 non-null     object
 7   duda             82 non-null     object
 8   adresse_postale  82 non-null     object
 9   usage            82 non-null     object
 10  etat             82 non-null     object
 11  usage2           82 non-null     object
 12  lehen_data       82 non-null     object
 13  beste            82 non-null     object
dtypes: int64(1), object(11), string(2)
memory usage: 9.1+ KB


irakurri berri duen zerrendako datuen tipologiak ikusi eta behar diren formatu informatikoetara itzuliko ditu

In [5]:
#%%capture
df = baserriak[baserriak['google_maps'].str.contains(',', na=False)]
#print(str(len(baserriak.index)) + " baserrietatik " + str(len(df.index)) + " dira ongi lokalizatuak")
# "loc" koordenatuen zatiketa bi partetan
df[['google_maps','lon']] = df['google_maps'].str.split(', ',expand=True)
# "loc" zena orain berrizendu eta "lat" (latitudea) deituko dut
df['lat']  = df['google_maps']
df = df.drop(['kontrola','en_mapa','beste','duda'], axis=1)
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['lon'] = pd.to_numeric(df['lon'], errors='coerce')
df = df.dropna(subset=['lat', 'lon'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 1 to 82
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   zenbakia         19 non-null     int64  
 1   izena            19 non-null     string 
 2   google_maps      19 non-null     string 
 3   auzoa            19 non-null     object 
 4   zonaldea         19 non-null     object 
 5   adresse_postale  19 non-null     object 
 6   usage            19 non-null     object 
 7   etat             19 non-null     object 
 8   usage2           19 non-null     object 
 9   lehen_data       19 non-null     object 
 10  lon              19 non-null     float64
 11  lat              19 non-null     float64
dtypes: float64(2), int64(1), object(7), string(2)
memory usage: 1.9+ KB


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
<ipython-input-5-03dc75c41c02>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lat']  = df['google_maps']


Geografi datuen solasa hasten da orain: mapeatzeko formato erabilgarrira pasatzen dira datuak,  sistema informatikoek uler diztaketen koordenatuak lortuz

In [6]:
from shapely.geometry import Point
df.loc[:, 'geometry'] = df.apply(lambda x: Point((float(x.lon), float(x.lat))), axis=1)

In [7]:
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.crs = {'init': 'epsg:4326'}
print(gdf.columns)

Index(['zenbakia', 'izena', 'google_maps', 'auzoa', 'zonaldea',
       'adresse_postale', 'usage', 'etat', 'usage2', 'lehen_data', 'lon',
       'lat', 'geometry'],
      dtype='object', name=0)


/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


Azken atal honek "sukaldatuak" dauden datuak hartu eta nahi bezala jarriko ditu mapa batean

In [8]:
import folium
#extraer vértices de marco
sw = gdf[['lat', 'lon']].min().values.tolist()
ne = gdf[['lat', 'lon']].max().values.tolist()
#determinar el fondo
tileado_name = "Stamen Terrain" #@param ["Stamen Toner", "Stamen Terrain", "OpenStreetMap", "Cartodbdark_matter"]
#crear el mapa con el fondo elegido
m = folium.Map(tiles=tileado_name)
#poner el marco
m.fit_bounds([sw, ne])

In [ ]:
from folium import plugins
minimap = plugins.MiniMap()
m.add_child(minimap)

In [10]:
#Baserri puntuak kargatu eta etiketak prestatu
for row in gdf.itertuples():
    folium.Marker([row.geometry.y, row.geometry.x],
                  icon=folium.Icon(
                      color='red', 
                      icon_color='white', 
                      icon='home', 
                      angle=0
                      ),
                  popup = folium.Popup(
                      "id <b>{}</b> <br><b>{}</b> <br>data: <b>{}</b><br>auzoa {} <br>zonaldea {} <br>adresse_postale {} <br>usage {} <br>etat {}<br> {}"
                      .format(
                          row.zenbakia, row.izena, row.lehen_data, row.auzoa, row.zonaldea,
                          row.adresse_postale, row.usage, row.etat, row.usage2))).add_to(m)

# Mapa ikusteko, *baserrimapa* hitzaren ezkerrean dagoen ▶ botoiari eman

In [11]:
from google.colab import drive
drive.mount('/content/drive')
m.save("/content/drive/My Drive/Proyectos/lmelosegi/index.html")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
